In [ ]:
import logging

import pypsa

logger = logging.getLogger(__name__)

n = pypsa.Network(snakemake.input["network"])

pgpfactor = float(snakemake.wildcards["pgpfactor"])

if snakemake.config["emulate_original_paper"]:
    # Following original paper model in 17_PGP.py only
    # apply the PGP cost modifier to electrolysis and fuel cell
    # fixed cost
    logger.warning(
        "'emulate_original_paper' enabled. "
        "Only applying pgpfactor modifier to electrolysis and fuel cell"
        "capital cost."
    )

    idx = n.df("Link").query("bus0 == 'hydrogen' or bus1 == 'hydrogen'").index
    n.df("Link").loc[idx, "capital_cost"] *= pgpfactor

else:
    # Apply PGP factor to all Link + Store components attached to the "hydrogen" bus
    n.df("Link").loc[
        n.df("Link").query("bus0 == 'hydrogen' or bus1 == 'hydrogen'").index,
        ["capital_cost", "marginal_cost"],
    ] *= pgpfactor
    n.df("Store").loc[
        n.df("Store").query("bus == 'hydrogen'").index,
        ["capital_cost", "marginal_cost"],
    ] *= pgpfactor

n.export_to_netcdf(snakemake.output["network"])